### Scrap fight records

Here, we go to the URLs of each fight and pull out their fight records. We'll format them separately, combine them, and save them as a big fat CSV.

In [15]:
import numpy as np
import pandas as pd

#scraping
import requests
from bs4 import BeautifulSoup

In [22]:
fighter_urls = pd.read_csv('.\Data\FighterURLs.csv')
fighter_urls.head(10)

,Name,URL
0,Aaron Riley,https://en.wikipedia.org/wiki/Aaron_Riley
1,Aaron Rosa,https://en.wikipedia.org/wiki/Aaron_Rosa
2,Aaron Simpson,https://en.wikipedia.org/wiki/Aaron_Simpson_(f...
3,Abel Trujillo,https://en.wikipedia.org/wiki/Abel_Trujillo
4,Adlan Amagov,https://en.wikipedia.org/wiki/Adlan_Amagov
5,Adriano Martins,https://en.wikipedia.org/wiki/Adriano_Martins_...
6,Akihiro Gono,https://en.wikipedia.org/wiki/Akihiro_Gono
7,Akira Corassani,https://en.wikipedia.org/wiki/Akira_Corassani
8,Al Iaquinta,https://en.wikipedia.org/wiki/Al_Iaquinta
9,Alan Belcher,https://en.wikipedia.org/wiki/Alan_Belcher


In [67]:
i = 2

fighter_page = requests.get(fighter_urls['URL'].iloc[i])
fighter_soup = BeautifulSoup(fighter_page.content, 'html.parser')

In [68]:
table = fighter_soup.findAll('table', class_='wikitable')


In [69]:
len(table)

2

In [70]:
raw_rows = []
for row in table[1].findAll('tr'):
    raw_rows.append(row.get_text().split('\n'))

raw_rows

[['',
  'Res.',
  'Record',
  'Opponent',
  'Method',
  'Event',
  'Date',
  'Round',
  'Time',
  'Location',
  'Notes',
  ''],
 ['',
  'Loss',
  '12–5',
  'Joshua Burkman',
  'KO (knee and punches)',
  'World Series of Fighting 2',
  '000000002013-03-23-0000March 23, 2013',
  '1',
  '3:04',
  'Atlantic City, New Jersey, United States',
  '',
  ''],
 ['',
  'Loss',
  '12–4',
  'Mike Pierce',
  'KO (punch)',
  'UFC on FX: Browne vs. Bigfoot',
  '000000002012-10-05-0000October 5, 2012',
  '2',
  '0:29',
  'Minneapolis, Minnesota, United States',
  '',
  ''],
 ['',
  'Win',
  '12–3',
  'Kenny Robertson',
  'Decision (unanimous)',
  'UFC on Fuel TV: Munoz vs. Weidman',
  '000000002012-07-11-0000July 11, 2012',
  '3',
  '5:00',
  'San Jose, California, United States',
  'Welterweight debut',
  ''],
 ['',
  'Loss',
  '11–3',
  'Ronny Markes',
  'Decision (split)',
  'UFC on Fuel TV: Sanchez vs. Ellenberger',
  '000000002012-02-15-0000February 15, 2012',
  '3',
  '5:00',
  'Omaha, Nebraska, U

In [71]:
pd.DataFrame(raw_rows[1:],columns=raw_rows[0])

,,Res.,Record,Opponent,Method,Event,Date,Round,Time,Location,Notes,
0,,Loss,12–5,Joshua Burkman,KO (knee and punches),World Series of Fighting 2,"000000002013-03-23-0000March 23, 2013",1,3:04,"Atlantic City, New Jersey, United States",,
1,,Loss,12–4,Mike Pierce,KO (punch),UFC on FX: Browne vs. Bigfoot,"000000002012-10-05-0000October 5, 2012",2,0:29,"Minneapolis, Minnesota, United States",,
2,,Win,12–3,Kenny Robertson,Decision (unanimous),UFC on Fuel TV: Munoz vs. Weidman,"000000002012-07-11-0000July 11, 2012",3,5:00,"San Jose, California, United States",Welterweight debut,
3,,Loss,11–3,Ronny Markes,Decision (split),UFC on Fuel TV: Sanchez vs. Ellenberger,"000000002012-02-15-0000February 15, 2012",3,5:00,"Omaha, Nebraska, United States",,
4,,Win,11–2,Eric Schafer,Decision (unanimous),UFC 136,"000000002011-10-08-0000October 8, 2011",3,5:00,"Houston, Texas, United States",,
5,,Win,10–2,Brad Tavares,Decision (unanimous),UFC 132,"000000002011-07-02-0000July 2, 2011",3,5:00,"Las Vegas, Nevada, United States",,
6,,Win,9–2,Mario Miranda,Decision (unanimous),UFC Fight Night: Nogueira vs. Davis,"000000002011-03-26-0000March 26, 2011",3,5:00,"Seattle, Washington, United States",,
7,,Loss,8–2,Mark Muñoz,Decision (unanimous),UFC 123,"000000002010-11-20-0000November 20, 2010",3,5:00,"Auburn Hills, Michigan, United States",,
8,,Loss,8–1,Chris Leben,TKO (punches),The Ultimate Fighter: Team Liddell vs. Team Or...,"000000002010-06-19-0000June 19, 2010",2,4:17,"Las Vegas, Nevada, United States",,
9,,Win,8–0,Tom Lawlor,Decision (split),UFC Fight Night: Maynard vs. Diaz,"000000002010-01-11-0000January 11, 2010",3,5:00,"Fairfax, Virginia, United States",Fight of the Night,
